In [ ]:
import numpy as np

funclist = { np.sum, np.count_nonzero }
sizelist = [ 10, 100, 1000, 10000, 100000 ]
dtypelist = [ np.int8, np.int16, np.int32, np.int64,
              np.float16, np.float32, np.float64, np.bool ]

for func in funclist:
    print(func.__name__)
    for dtype in dtypelist:
        print(f"dtype: {dtype.__name__}")
        for size in sizelist:
            print(f"size: {size}")
            ndarray = np.zeros((size,), dtype=dtype)
            %timeit func(ndarray)
        print()
    print()

count_nonzero
dtype: int8
size: 10
298 ns ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 100
292 ns ± 9.41 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 1000
396 ns ± 12.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 10000
682 ns ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 100000
3.63 μs ± 234 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

dtype: int16
size: 10
291 ns ± 3.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 100
314 ns ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 1000
429 ns ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 10000
966 ns ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
size: 100000
6.34 μs ± 152 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

dtype: int32
size: 10
288 ns ± 3.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,

In [ ]:
from marubatsu import Marubatsu, Board
import numpy as np

class NpBoolBoard(Board):
    EMPTY = -1
    CIRCLE = 0
    CROSS = 1
    MARK_TABLE = {
        EMPTY: Marubatsu.EMPTY_STR,
        CIRCLE: Marubatsu.CIRCLE_STR,
        CROSS: Marubatsu.CROSS_STR,
        Marubatsu.DRAW: Marubatsu.DRAW,        
    }
    
    def __init__(self, board_size=3, count_linemark=False):
        self.BOARD_SIZE = board_size
        self.count_linemark = count_linemark
        self.board = np.full((2, self.BOARD_SIZE, self.BOARD_SIZE), False)
        if self.count_linemark:
            self.rowcount = {
                self.CIRCLE: [0] * self.BOARD_SIZE,
                self.CROSS: [0] * self.BOARD_SIZE,
            }
            self.colcount = {
                self.CIRCLE: [0] * self.BOARD_SIZE,
                self.CROSS: [0] * self.BOARD_SIZE,
            }
            self.diacount = {
                self.CIRCLE: [0] * 2,
                self.CROSS: [0] * 2,
            }

    def getmark_by_move(self, move):
        if self.board[(self.CIRCLE, ) + move]:
            return self.CIRCLE
        elif self.board[(self.CROSS, ) + move]:
            return self.CROSS
        else:
            return self.EMPTY
        
    def getmark(self, x, y):
        if self.board[(self.CIRCLE, x, y)]:
            return self.CIRCLE
        elif self.board[(self.CROSS, x, y)]:
            return self.CROSS
        else:
            return self.EMPTY

    def setmark_by_move(self, move, mark):
        x, y = move
        if self.count_linemark:
            if mark != self.EMPTY:
                diff = 1
                changedmark = mark
            else:
                diff = -1
                changedmark = self.getmark(x, y)
            self.colcount[changedmark][x] += diff
            self.rowcount[changedmark][y] += diff
            if x == y:
                self.diacount[changedmark][0] += diff
            if x + y == self.BOARD_SIZE - 1:
                self.diacount[changedmark][1] += diff
        if mark == self.EMPTY:
            if self.board[self.CIRCLE, x, y]:
                self.board[self.CIRCLE, x, y] = False
            else:
                self.board[self.CROSS, x, y] = False
        else:
            self.board[mark, x, y] = True

    def xy_to_move(self, x, y):
        return (x, y)

    def move_to_xy(self, move):
        return move  

    def calc_legal_moves(self):
        legal_moves = [(x, y) for y in range(self.BOARD_SIZE) 
                            for x in range(self.BOARD_SIZE)
                            if self.getmark(x, y) == self.EMPTY]
        return legal_moves    

    def board_to_str(self):
        txt = ""
        for x in range(self.BOARD_SIZE):
            for y in range(self.BOARD_SIZE):
                txt += self.MARK_TABLE[self.getmark(x, y)]
        return txt

    def board_to_hashable(self):
        return self.board.tobytes()

    def judge(self, last_turn, last_move, move_count):
        if move_count < self.BOARD_SIZE * 2 - 1:
            return Marubatsu.PLAYING
        # 直前に着手を行ったプレイヤーの勝利の判定
        if self.is_winner(last_turn, last_move):
            return last_turn
        # 引き分けの判定
        elif move_count == self.BOARD_SIZE ** 2:
            return Marubatsu.DRAW
        # 上記のどれでもなければ決着がついていない
        else:
            return Marubatsu.PLAYING  
        
    def is_winner(self, player, last_move):
        x, y = last_move
        if self.count_linemark:
            if self.rowcount[player][y] == self.BOARD_SIZE or \
            self.colcount[player][x] == self.BOARD_SIZE:
                return True
            # 左上から右下方向の判定
            if x == y and self.diacount[player][0] == self.BOARD_SIZE:
                return True
            # 右上から左下方向の判定
            if x + y == self.BOARD_SIZE - 1 and \
                self.diacount[player][1] == self.BOARD_SIZE:
                return True
        else:
            if np.count_nonzero(self.board[player, x, :]) == self.BOARD_SIZE  or \
            np.count_nonzero(self.board[player, :, y]) == self.BOARD_SIZE:
                return True
            # 左上から右下方向の判定
            if x == y and np.count_nonzero(np.diag(self.board[player])) == self.BOARD_SIZE:
                return True
            # 右上から左下方向の判定
            if x + y == self.BOARD_SIZE - 1 and \
                np.count_nonzero(np.diag(np.fliplr(self.board[player]))) == self.BOARD_SIZE:
                return True
        
        # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
        return False    

    def count_markpats(self, turn, last_turn):
        pass

    def count_marks(self, linedata, turn, last_turn):
        pass

    def calc_same_hashables(self):
        pass

In [3]:
from ai import ai_match, ai2
import random

random.seed(0)
ai_match(ai=[ai2, ai2], match_num=50000, mbparams={"boardclass": NpBoolBoard})
random.seed(0)
ai_match(ai=[ai2, ai2], match_num=50000, mbparams={"boardclass": NpBoolBoard, "count_linemark": True})

ai2 VS ai2


  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [00:06<00:00, 7587.97it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai2 VS ai2


100%|██████████| 50000/50000 [00:07<00:00, 6367.55it/s]

count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%



[('count',
  [{'win': 29454, 'lose': 14352, 'draw': 6194},
   {'win': 14208, 'lose': 29592, 'draw': 6200},
   {'win': 43662, 'lose': 43944, 'draw': 12394}],
  '7d'),
 ('ratio',
  [{'win': 0.58908, 'lose': 0.28704, 'draw': 0.12388},
   {'win': 0.28416, 'lose': 0.59184, 'draw': 0.124},
   {'win': 0.43662, 'lose': 0.43944, 'draw': 0.12394}],
  '7.1%')]